In [71]:
from Data_Enrichment import get_features

RAW_DIR = "data/raw"
df_feats, feature_cols = get_features(RAW_DIR)
df_feats = df_feats[(df_feats['season_end_year'] != 2026)]

/workspaces/Machine-learning/Data_Enrichment.py:280: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["ballon_dor_winner"] = df["ballon_dor_winner"].fillna(False).astype(bool)


In [72]:
# 🔹 Filtro limpio manteniendo identificador y variables relevantes + age y age_penalty
df_model = df_feats.filter(
    regex=r'(player_id|player_name|lag1|delta|_w$|height|main_position|position|season_end_year|ballon_dor_winner|age|age_penalty)'
).copy()

df_model = df_model.dropna()


In [73]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [74]:
# Target
y = df_model['ballon_dor_winner'].values

# Features (excluyendo columnas no numéricas o irrelevantes)
X = df_model.drop(columns=['ballon_dor_winner', 'player_name']).values


In [75]:
# Seleccionar solo columnas numéricas
X_numeric = df_model.select_dtypes(include=['int64', 'float64',"bool"])


In [76]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)


In [77]:
np.random.seed(30) #we fix the seed so we can reproduce the data any time
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_numeric, X_numeric["ballon_dor_winner"], test_size=0.3, random_state=42)

In [78]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit Lasso regression model
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

,alpha,0.1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [79]:
y_pred = lasso.predict(X_test)
print(y_pred.mean())

0.0003313452617627567


In [80]:
# Model Score
print("Model Score: ", lasso.score(X_test, y_test))

Model Score:  -5.5236411842463085e-05


chat help me this part

In [81]:
import pandas as pd
import numpy as np

# Target
y = df_model['ballon_dor_winner'].values  # 1 si ganó, 0 si no

# Features
# Excluimos target y player_name
X = df_model.drop(columns=['ballon_dor_winner', 'player_name']).copy()

# Detectar columnas no numéricas
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:", categorical_cols)

# Codificar variables categóricas
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
print("Shape after encoding:", X_encoded.shape)

# Convertimos a numpy array
X_values = X_encoded.values


Categorical columns: ['position', 'main_position']
Shape after encoding: (30180, 47)


In [82]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_values, y, test_size=0.2, random_state=42, stratify=y
)


In [83]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [84]:
from sklearn.linear_model import LogisticRegression

log_lasso = LogisticRegression(
    penalty='l1',      # Lasso
    solver='saga',     # necesario para L1
    class_weight='balanced',  # ayuda con clases desbalanceadas
    max_iter=10000,
    random_state=42
)

# Entrenamos
log_lasso.fit(X_train_scaled, y_train)


,penalty,'l1'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'saga'
,max_iter,10000
,multi_class,'deprecated'


In [85]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

# Predicciones de probabilidad
y_pred_prob = log_lasso.predict_proba(X_test_scaled)[:,1]

# Predicciones binarias con threshold 0.5
y_pred = (y_pred_prob >= 0.5).astype(int)

# Métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_prob))


Accuracy: 0.9983432736911863
Precision: 0.0
Recall: 0.0
ROC AUC: 0.9617998011269473


In [86]:
# Asegurarse de que df_feats y y tengan el mismo tamaño
min_len = min(len(df_feats), len(y))
df_feats_aligned = df_feats.iloc[:min_len].copy()
y_aligned = y[:min_len]

# Ahora sí split
_, X_test_idx, _, _ = train_test_split(
    np.arange(len(df_feats_aligned)),
    y_aligned,
    test_size=0.2,
    random_state=42,
    stratify=y_aligned
)

# DataFrame de predicciones
df_preds = pd.DataFrame({
    "player_name": df_feats_aligned.iloc[X_test_idx]["player_name"].values,
    "pred_prob": y_pred_prob
})

# Top 30 jugadores
top_players = df_preds.sort_values(by='pred_prob', ascending=False).head(30)
print(top_players)


                           player_name  pred_prob
947              Harry Wilson (279455)   0.999452
5972             Harry Wilson (279455)   0.999381
5776          Anthony Mandréa (290559)   0.987635
3743            Daler Kuzyaev (279570)   0.983146
3976             Bruno Jordão (415646)   0.947102
1035            Mohamed Salah (148455)   0.821950
3468                 Juan Mata (44068)   0.678777
2821             Divock Origi (148368)   0.598554
4484        Giuseppe Di Serio (532040)   0.484677
1048             Divock Origi (148368)   0.416007
5431              Enzo Tchato (842338)   0.404269
1892             Nils Seufert (279064)   0.399396
757         Dimitri Foulquier (170472)   0.375742
4687              Sékou Koïta (402010)   0.330977
5279         Thomas Rodríguez (402581)   0.301191
2690               Nathan Aké (177476)   0.282571
1785              Nick Powell (134424)   0.274549
4288          Rigoberto Rivas (451353)   0.190734
1733            Raúl Bobadilla (51356)   0.179019


In [88]:
# Coeficientes del modelo
coef = log_lasso.coef_[0]
feature_names = X_encoded.columns

# Ordenar por importancia absoluta
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coef': coef
}).sort_values(by='coef', key=abs, ascending=False)

print(feature_importance.head(30))


                                   feature      coef
30        position_Attack - Centre-Forward -1.894074
2                                      age  1.590542
7                           g_per90_z_lag1  0.933989
15                  discipline_rate_z_lag1 -0.927433
8                          g_per90_z_delta  0.888585
32          position_Attack - Right Winger  0.833961
18                       pen_share_z_delta -0.668123
21                   matches_played_z_lag1  0.607483
3                          season_end_year -0.581370
44                  main_position_Defender  0.514901
24                               g_per90_w -0.509106
16                 discipline_rate_z_delta -0.408559
1                                   height -0.403550
41  position_Midfield - Defensive Midfield  0.377375
39  position_Midfield - Attacking Midfield -0.355101
22                  matches_played_z_delta  0.334444
25                               a_per90_w  0.316968
6                         ga_per90_z_delta  0.